In [78]:
#pip install Pillow

In [79]:
#pip install selenium

In [80]:
pip install hashlib

  Using cached https://files.pythonhosted.org/packages/74/bb/9003d081345e9f0451884146e9ea2cff6e4cc4deac9ffd4a9ee98b318a49/hashlib-20081119.zip
Note: you may need to restart the kernel to use updated packages.


    ERROR: Complete output from command python setup.py egg_info:
    ERROR: Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\mrudh\Anaconda3\lib\site-packages\setuptools\__init__.py", line 18, in <module>
        import setuptools.version
      File "C:\Users\mrudh\Anaconda3\lib\site-packages\setuptools\version.py", line 1, in <module>
        import pkg_resources
      File "C:\Users\mrudh\Anaconda3\lib\site-packages\pkg_resources\__init__.py", line 36, in <module>
        import email.parser
      File "C:\Users\mrudh\Anaconda3\lib\email\parser.py", line 12, in <module>
        from email.feedparser import FeedParser, BytesFeedParser
      File "C:\Users\mrudh\Anaconda3\lib\email\feedparser.py", line 27, in <module>
        from email._policybase import compat32
      File "C:\Users\mrudh\Anaconda3\lib\email\_policybase.py", line 9, in <module>
        from email.utils import _has_surrogates
      File "C:\Users\mrudh\Anaconda3\lib\e

In [81]:
import selenium
import os
# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
DRIVER_PATH = r'C:\\Users\\mrudh\\Desktop\\Scraping\\chromedriver_win32\\chromedriver.exe'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)
#driver = webdriver.Chrome('C:\Webdrivers') 

In [82]:
from selenium import webdriver
import time
#from time import sleep

def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [83]:
import requests
import io
from PIL import Image
import hashlib
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [85]:

def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=100):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=1)
        
    for elem in res:
        persist_image(target_folder,elem)

In [86]:
#driver=webdriver.Chrome(executable_path="C:\Program Files (x86)\Google\Chrome\Application\chrome.exe")
#driver.get("http://www.python.org")
search_term='reserach paper IEEE'
search_and_download(
    search_term=search_term,
    driver_path=DRIVER_PATH
)

Found: 200 search results. Extracting links from 0:200
Found: 100 image links, done!
SUCCESS - saved https://ieeexplore.ieee.org/xploreAssets/images/absImages/04199177.png - as ./images\reserach_paper_ieee\5115f34126.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ9M7W6lBPziFVrELWalWlGslZPM4Zhy5Swe5L0oVBY1__JCRhr&usqp=CAU - as ./images\reserach_paper_ieee\f18687824b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSNOXzRIO3EGDiK_iAo47awaO6YZDXJgYGGEFKHiyDjWGY-EMCr&usqp=CAU - as ./images\reserach_paper_ieee\adec506632.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRfOphB52qoJ3agnrBa-TJhRmUXUeXlc52DQs6P5xrTI8BwX__l&usqp=CAU - as ./images\reserach_paper_ieee\450dbc4f26.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSp3rOp7FgCRJxobOEsPbCYr2J73UODoJpvYPpcZ0uUNTQ3g7f-&usqp=CAU - as ./images\reserach_paper_ieee\058218548c.jpg
SUCCESS - saved https://image.slidesharecdn.com/ieeepap

SUCCESS - saved https://ieeexplore.ieee.org/xploreAssets/images/absImages/00503157.png - as ./images\reserach_paper_ieee\357850b27e.jpg
SUCCESS - saved https://eagereyes.org/wp-content/uploads/2015/10/tableau-vis-papers-teaser.jpg - as ./images\reserach_paper_ieee\3091d4d7eb.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ8QyPM-XHNY_lTs8kha6z7JuCgo7uvnByrzl6GeHoObMq4NXPy&usqp=CAU - as ./images\reserach_paper_ieee\ccdde499d0.jpg
SUCCESS - saved https://ieeexplore.ieee.org/xploreAssets/images/absImages/00787368.png - as ./images\reserach_paper_ieee\c83444d4fd.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQcdTOHtfruYbTDvZrZzS2G3LRvNjBkBJ74DwFVv7IHITPhahjJ&usqp=CAU - as ./images\reserach_paper_ieee\abf3df1838.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcReFx6mIB2tyGmCrZ_1aGwu6rYB7dV2D79E2srwrey4KrDfrKn_&usqp=CAU - as ./images\reserach_paper_ieee\fd421e84ba.jpg
SUCCESS - saved https://encrypted-tbn0.gs

SUCCESS - saved https://ieee-ceda.org/sites/ieeeceda/files/styles/cc_container/public/pdfpreview/6946-TCAD%20SI%20CFP%20V4.png?itok=dawkvQnV - as ./images\reserach_paper_ieee\77629e9dee.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ7uNjrK0ddoVrgA7WR7cLMDvkAtQUZNfOiIAQlC2ZDSWRr_dH1&usqp=CAU - as ./images\reserach_paper_ieee\87e49b997c.jpg
SUCCESS - saved https://i1.rgstatic.net/publication/3188260_Preliminary_Guidelines_for_Empirical_Research_in_Software_Engineering/links/551299300cf268a4aaea944a/largepreview.png - as ./images\reserach_paper_ieee\ae8c763bdd.jpg
